# Competitive model
My personal entry for the SDOBenchmark consists of a model trained on a categorization of the label *peak_flux*. This model performs predictions with a **True Skill Statistic** of **0.34** and a converted **Mean Absolute Error** of **3.9e-5**. In other words, it is better than random or constant predictions, but is far from any reliable prediction.
I use the *HMI magnetogram* and *AIA 304, 131 and 1700* as inputs, alongside the date of the event.

In [3]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = "3"
import tensorflow as tf
# limit memory usage
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from keras import backend as K
K.set_session(sess)
from keras.models import load_model
import numpy as np
from utils.statistics import *
from utils.keras_generator import SDOBenchmarkGenerator
from utils.statistics import *

In [2]:
base_path = '/media/all/D4/publish/'

# Parameters
params = {'dim': (4, 256, 256, 4),
          'batch_size': 16,
          'channels': ['magnetogram', '304', '131', '1700'],
          'shuffle': False}

validation_generator = SDOBenchmarkGenerator(os.path.join(base_path, 'test'), **params)


def converted_mae(y_true, y_pred):
    conv_y_true = 10.**(K.cast(K.argmax(y_true, axis=-1), K.floatx())-9.) * 5.
    conv_y_pred = 10.**(K.cast(K.argmax(y_pred, axis=-1), K.floatx())-9.) * 5.
    return K.mean(K.abs(conv_y_pred - conv_y_true), axis=-1)

model = load_model('/home/roman/Documents/GitHub/SDOBenchmark/notebooks/models/best_binary', custom_objects={'converted_mae': converted_mae})
# model.summary()

Y_pred = model.predict_generator(validation_generator,
                        steps = len(validation_generator),
                        use_multiprocessing=True,
                        max_queue_size=params['batch_size'],
                        workers=params['batch_size'] * 2,
                        verbose=2
                        )

## Evaluation

In [88]:
Y_pred_flux = 10. ** (np.argmax(Y_pred, axis=-1)-9.)*5
Y_val = np.asarray(validation_generator.data.iloc[validation_generator.indexes]['peak_flux'])

print(f'Mean absolute error:  {np.mean(np.abs(Y_val-Y_pred_flux))}')
print('TSS: ' + str(true_skill_statistic(Y_val, Y_pred_flux)))
print(f'(categorical_accuracy: {np.mean(np.equal(np.floor(np.log10(Y_val)+9).astype(int), np.argmax(Y_pred, axis=-1)).astype(np.float32))})')

Mean absolute error:  3.89457729693513e-05
TSS: 0.34024166660413646
(categorical_accuracy: 0.47488316893577576)


#### Performance on big flares (X-flares):

In [90]:
X_index = np.where(Y_val >= 1e-4)[0]
print(f'{np.sum((Y_pred_flux[X_index]>=1e-4).astype(int))} of {len(X_index)} were correctly classified as X-flares')
print(f'{len(np.where(np.logical_and(Y_val < 1e-4, Y_pred_flux >= 1e-4))[0])} were wrongly classified as X-flares')
print(f'(categorical_accuracy: {np.mean(np.equal(np.floor(np.log10(Y_val[X_index])+9).astype(int), np.argmax(Y_pred[X_index], axis=-1)).astype(np.float32))})')

12 of 46 were correctly classified as X-flares
168 were wrongly classified as X-flares
(categorical_accuracy: 0.260869562625885)


#### Would the model have predicted the large flare of September 2017?

In [91]:
september_index = np.where(np.logical_and(validation_generator.data.iloc[validation_generator.indexes].index.str.startswith('12673'), Y_val >= 1e-4))
validation_generator.data['peak_class'] = [flux_to_class(f) for f in validation_generator.data['peak_flux']]
validation_generator.data['predicted_class'] = [flux_to_class(float(f)) for f in Y_pred_flux]
validation_generator.data.iloc[september_index]

,start,end,peak_flux,flip,predicted,peak_class,predicted_class
12673_2017_09_05_12_02_01_0,2017-09-05 00:02:01,2017-09-05 12:02:01,0.001094,False,5.000000e-05,X11,M5
12673_2017_09_05_12_02_01_1,2017-09-06 00:02:00,2017-09-06 12:02:00,0.001094,False,5.000000e-05,X11,M5
12673_2017_09_05_09_10_01_0,2017-09-04 21:10:01,2017-09-05 09:10:01,0.000259,False,5.000000e-06,X3,C5
12673_2017_09_05_09_10_01_1,2017-09-05 21:10:00,2017-09-06 09:10:00,0.000259,False,5.000000e-05,X3,M5
12673_2017_09_09_16_06_01_0,2017-09-09 04:06:01,2017-09-09 16:06:01,0.000965,False,5.000000e-07,X10,B5
12673_2017_09_06_14_36_01_0,2017-09-06 02:36:01,2017-09-06 14:36:01,0.000153,False,5.000000e-05,X2,M5
12673_2017_09_06_14_36_01_1,2017-09-07 02:36:00,2017-09-07 14:36:00,0.000153,False,5.000000e-05,X2,M5
12673_2017_09_05_12_02_01_0_copy,2017-09-05 00:02:01,2017-09-05 12:02:01,0.001094,True,5.000000e-05,X11,M5
12673_2017_09_05_12_02_01_1_copy,2017-09-06 00:02:00,2017-09-06 12:02:00,0.001094,True,5.000000e-05,X11,M5
12673_2017_09_05_09_10_01_0_copy,2017-09-04 21:10:01,2017-09-05 09:10:01,0.000259,True,5.000000e-06,X3,C5


No, it would have predicted a large flare (M), not a very large flare (X+).